# Correlation Bootstrap Test (Null threshold)
Derives 95% |corr| under permutation to justify falsifier cutoff.

In [0]:
import pandas as pd, numpy as np
from pathlib import Path
lat = pd.read_csv('data/processed/latents.csv')
regions = lat['region'].unique()
results = []
for r in regions:
    sub = lat[lat['region']==r].sort_values('year')
    if len(sub) < 10: continue
    sigma = sub['sigma'].to_numpy()
    rho = sub['rho'].to_numpy()
    ds = np.diff(sigma)
    rr = rho[1:]
    if len(ds)==len(rr):
        c = np.corrcoef(rr, -ds)[0,1]
        results.append({'region':r, 'corr':float(c)})
pd.DataFrame(results).to_csv('results/corr_empirical.csv', index=False)
B=500; null=[]; rng=np.random.default_rng(0)
for r in regions:
    sub = lat[lat['region']==r].sort_values('year')
    if len(sub) < 10: continue
    sigma = sub['sigma'].to_numpy(); rho=sub['rho'].to_numpy()
    ds = np.diff(sigma); rr = rho[1:]
    for b in range(B):
        perm = rng.permutation(len(rr))
        null.append(np.corrcoef(rr[perm], -ds)[0,1])
thr = float(np.percentile(np.abs(null),95))
Path('results').mkdir(exist_ok=True)
Path('results/corr_bootstrap.json').write_text(str({'abs_corr_95pct':thr}), encoding='utf-8')
print('Null 95% |corr| threshold:', thr)
